# Gauge transformation of the gauge links in a LC coordinate lattice

## Step 0

### Writting the fields from files

In [2]:
import os

os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = 'su3'

import curraun.su as su
import numpy as np
import pickle

Using SU(3)
Using Numba
Using double precision


/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# We read the fields

fields = np.load('gauge_links_fields_glasma_test.npz')
ux = fields['ux']

with open('parameters_gauge_transf_glasma_test.pickle', 'rb') as f:
    param = pickle.load(f)

In [4]:
print(np.shape(ux))

(64, 16384, 9)


In [5]:
# We get the relevant parameters

N = param['N']
Dg = param['DG']

t_steps = np.shape(ux)[0]

In [6]:
# We reshape the transverse component into a N*N matrix

ux_r = ux.reshape((t_steps, N, N, Dg))

print(np.shape(ux_r))

(64, 128, 128, 9)


## Step 1

### We write the relevant magnitude in LC coordinates

In [7]:
# We define a function that, given a set of gauge links, returns the values over the x^+ axis

def get_xplus_axis_links (u):
    uminus = su.GROUP_TYPE(np.zeros((t_steps, N, Dg)))
    for t in range (t_steps):
        uminus[t, :, :] = u[t,t,:,:]
    uplus = np.apply_along_axis(su.dagger, 2, uminus)
    return uplus, uminus

In [8]:
u_plus_temp, u_minus_temp = get_xplus_axis_links(ux_r)

print(np.shape(u_plus_temp))
print(np.shape(u_minus_temp))

(64, 128, 9)
(64, 128, 9)


/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/core/typed_passes.py:329: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../curraun/su3.py", line 347:
@myjit
def dagger(a):
^

  warnings.warn(errors.NumbaPerformanceWarning(msg,


## Step 2

### We gauge transform the relevant quantity to LC gauge

In [9]:
# We construct a function that gives the gauge operator at a given point of the x^+ axis

def gauge_operator(u, xplus):
    v = su.GROUP_TYPE(np.array([su.unit() for i in range(N)]))
    for t in range(xplus):
        buffer1 = v
        aux = u[t, 2*xplus-t, :, :]
        buffer2 = np.apply_along_axis(su.dagger, 1, aux)
        v = np.array([su.mul(buffer1[i, :], buffer2[i, :]) for i in range(N)])
    return v

In [10]:
# We construct an array with the gauge operator at every x^+ step

v_LC = su.GROUP_TYPE(np.zeros((t_steps, N, Dg)))

for t in range (t_steps):
    v_LC[t,:,:] = gauge_operator(ux_r, t)

print(np.shape(v_LC))

/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/core/typed_passes.py:329: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../curraun/su3.py", line 371:
@myjit
def unit():
^

  warnings.warn(errors.NumbaPerformanceWarning(msg,


/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/core/typed_passes.py:329: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../curraun/su3.py", line 119:
@myjit
def mul(a, b):
^

  warnings.warn(errors.NumbaPerformanceWarning(msg,


(64, 128, 9)


In [17]:
# We construct a function that performs the gauge transformation

def act_plus_link (u,v):
    v_h = np.apply_along_axis(su.dagger, 2, v)
    buffer1 = np.array([[su.mul(v[t+1, i, :], u[t, i, :]) for i in range(N)] for t in range(t_steps-1)]) #TODO: What happens with the last step?
    result =  np.array([[su.mul(buffer1[t, i, :], v_h[t, i, :]) for i in range(N)] for t in range(t_steps-1)])
    return result

In [18]:
# We transform the plus link

uplus_LC = act_plus_link(u_plus_temp, v_LC)

print(np.shape(uplus_LC))

(63, 128, 9)
